In [19]:

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score


def load_transactions(file_path):
    """
    Load transactions from a file into a list of sets.

    Args:
    - file_path (str): The path to the file containing the transactions.

    Returns:
    - List[Set]: A list of transactions where each transaction is a set of items.
    """
    transactions = []
    with open(file_path, 'r') as file:
        for line in file:
            transaction = line.strip().split()
            transactions.append(transaction)
    return transactions

transactions = load_transactions('../data/auto.num.txt')
transactions

[['3',
  '14',
  '28',
  '29',
  '31',
  '35',
  '39',
  '41',
  '45',
  '49',
  '55',
  '60',
  '62',
  '70',
  '76',
  '82',
  '87',
  '95',
  '100',
  '105',
  '109',
  '112',
  '120',
  '126',
  '127',
  '134'],
 ['2',
  '9',
  '28',
  '29',
  '32',
  '36',
  '39',
  '41',
  '43',
  '47',
  '52',
  '58',
  '62',
  '70',
  '76',
  '81',
  '87',
  '93',
  '99',
  '107',
  '109',
  '114',
  '121',
  '126',
  '127',
  '135'],
 ['3',
  '23',
  '28',
  '29',
  '31',
  '35',
  '38',
  '41',
  '44',
  '48',
  '53',
  '59',
  '62',
  '71',
  '76',
  '82',
  '87',
  '95',
  '98',
  '106',
  '109',
  '112',
  '120',
  '123',
  '127',
  '134'],
 ['4',
  '17',
  '28',
  '29',
  '32',
  '36',
  '39',
  '41',
  '43',
  '47',
  '53',
  '58',
  '62',
  '70',
  '76',
  '81',
  '87',
  '93',
  '99',
  '106',
  '109',
  '115',
  '121',
  '126',
  '127',
  '136'],
 ['3',
  '17',
  '28',
  '30',
  '31',
  '35',
  '39',
  '41',
  '44',
  '49',
  '53',
  '58',
  '62',
  '70',
  '76',
  '82',
  '91',
  '93

In [20]:
pip install mlxtend


Note: you may need to restart the kernel to use updated packages.


In [21]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

In [22]:

# Binarisation (Étape de pré-traitement )
# Transformation en un format one-hot encodé
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_transactions = pd.DataFrame(te_ary, columns=te.columns_)

print("Aperçu des données binarisées :")
print(df_transactions)

Aperçu des données binarisées :
         1     10    100    101    102    103    105    106    107    108  \
0    False  False   True  False  False  False   True  False  False  False   
1    False  False  False  False  False  False  False  False   True  False   
2    False  False  False  False  False  False  False   True  False  False   
3    False  False  False  False  False  False  False   True  False  False   
4    False  False  False   True  False  False   True  False  False  False   
..     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
200  False  False   True  False  False  False  False  False   True   True   
201  False  False  False  False   True  False  False   True  False  False   
202  False  False  False  False   True  False   True  False  False  False   
203  False  False  False  False  False  False  False   True  False  False   
204   True  False  False  False   True  False  False   True  False  False   

     ...     90     91     92     93     94

In [23]:
# 1. Extraction des itemsets fréquents avec un seuil de support bas 
# Nous choisissons un seuil bas (ex: 33% de support) pour générer un pool conséquent.
min_support_seuil_bas = 0.33 

frequent_itemsets = fpgrowth(df_transactions, min_support=min_support_seuil_bas, use_colnames=True)

print(f"\nNombre d'itemsets fréquents trouvés (seuil bas) : {len(frequent_itemsets)}")
print(frequent_itemsets.head())

# 2. Génération des règles d'association à partir des itemsets fréquents
# Nous pouvons utiliser une métrique comme la 'confidence' ou le 'lift' pour filtrer
# un peu, mais l'idée est de garder un grand pool P.
# Nous utilisons un seuil de 'confidence' bas pour cet exemple.
P_pool_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)


# 3. Calcul des métriques 
# mlxtend nous donne déjà : 'support', 'confidence', 'lift'
# Ajoutons 'length' (longueur du motif, antécédent + conséquent) et 'coverage'
# Note : 'coverage' est souvent défini comme le 'antecedent support'

P_pool_rules['length'] = P_pool_rules['antecedents'].apply(lambda x: len(x)) + \
                         P_pool_rules['consequents'].apply(lambda x: len(x))

# 'coverage' est déjà présent sous le nom 'antecedent support'
P_pool_rules.rename(columns={'antecedent support': 'coverage'}, inplace=True)

# Réorganisation des colonnes pour la clarté
P_pool_rules = P_pool_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift', 'coverage', 'length']]

print(f"\nPool P de {len(P_pool_rules)} motifs (règles) généré :")
print(P_pool_rules)


Nombre d'itemsets fréquents trouvés (seuil bas) : 1126
    support itemsets
0  0.985366     (41)
1  0.902439     (28)
2  0.819512     (29)
3  0.775610     (76)
4  0.721951     (70)

Pool P de 27915 motifs (règles) généré :
      antecedents consequents   support  confidence      lift  coverage  \
0            (28)        (41)  0.887805    0.983784  0.998394  0.902439   
1            (41)        (28)  0.887805    0.900990  0.998394  0.985366   
2            (29)        (41)  0.804878    0.982143  0.996729  0.819512   
3            (41)        (29)  0.804878    0.816832  0.996729  0.985366   
4            (29)        (28)  0.785366    0.958333  1.061937  0.819512   
...           ...         ...       ...         ...       ...       ...   
27910    (90, 41)        (28)  0.443902    1.000000  1.108108  0.443902   
27911    (28, 41)        (90)  0.443902    0.500000  1.090426  0.887805   
27912        (90)    (28, 41)  0.443902    0.968085  1.090426  0.458537   
27913        (40)        (

In [24]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Copie du pool pour ne pas modifier l'original
P_sampling = P_pool_rules.copy()

# Normalisation des métriques (exemple : lift et confidence)
metrics_to_normalize = ['lift', 'confidence']
P_sampling[metrics_to_normalize] = scaler.fit_transform(P_sampling[metrics_to_normalize])

# Définition d'un score composite simple (pondération 60% lift, 40% confidence)
# C'est ici que vous pouvez tester différentes stratégies 
w_lift = 0.6
w_conf = 0.4
P_sampling['composite_score'] = (w_lift * P_sampling['lift']) + (w_conf * P_sampling['confidence'])

# Ajout du poids de feedback (initialisé à 1) 
P_sampling['feedback_weight'] = 1.0

# Le poids final pour l'échantillonnage sera (score * feedback)
P_sampling['final_sampling_weight'] = P_sampling['composite_score'] * P_sampling['feedback_weight']


print("\nPool P avec scores composites et poids de feedback :")
print(P_sampling[['antecedents', 'consequents', 'composite_score', 'feedback_weight', 'final_sampling_weight']].head())


Pool P avec scores composites et poids de feedback :
  antecedents consequents  composite_score  feedback_weight  \
0        (28)        (41)         0.428930              1.0   
1        (41)        (28)         0.362695              1.0   
2        (29)        (41)         0.427006              1.0   
3        (41)        (29)         0.294757              1.0   
4        (29)        (28)         0.431896              1.0   

   final_sampling_weight  
0               0.428930  
1               0.362695  
2               0.427006  
3               0.294757  
4               0.431896  


In [25]:
def weighted_sampling(df_pool, k, with_replacement=False):
    """
    Effectue un échantillonnage pondéré sur le pool de motifs.
    
    """
    
    # Normalisation des poids pour qu'ils somment à 1 (transforme en probabilités)
    weights = df_pool['final_sampling_weight']
    probabilities = weights / weights.sum()
    
    # Gestion du cas où tous les poids sont à zéro
    if probabilities.sum() == 0:
        print("Avertissement : Tous les poids sont à zéro. Échantillonnage uniforme.")
        probabilities = None # numpy.random.choice passera en uniforme
        
    # Utilisation de numpy.random.choice pour l'échantillonnage pondéré
    sampled_indices = np.random.choice(
        df_pool.index,
        size=k,
        replace=with_replacement,
        p=probabilities
    )
    
    return df_pool.loc[sampled_indices]

# Exemple : Échantillonner k=3 motifs SANS remise
k_sample = 3
sample_1 = weighted_sampling(P_sampling, k=k_sample, with_replacement=False)

print(f"\n--- Premier échantillon (k={k_sample}) ---")
print(sample_1[['antecedents', 'consequents', 'composite_score', 'final_sampling_weight']])


--- Premier échantillon (k=3) ---
             antecedents    consequents  composite_score  \
18321  (76, 28, 127, 39)  (41, 62, 126)         0.702869   
7778   (29, 28, 70, 127)       (76, 39)         0.601780   
10981     (109, 127, 70)           (41)         0.447944   

       final_sampling_weight  
18321               0.702869  
7778                0.601780  
10981               0.447944  


In [ ]:
def update_weights_with_feedback(df_pool, liked_indices, disliked_indices):
    """
    Met à jour les 'feedback_weight' du pool basé sur le feedback.
    
    """
    new_pool = df_pool.copy()
    
    # Facteurs d'augmentation/diminution (à calibrer)
    LIKE_FACTOR = 1.5  # Augmente le poids de 50%
    DISLIKE_FACTOR = 0.5 # Diminue le poids de 50%
    
    if not liked_indices.empty:
        new_pool.loc[liked_indices, 'feedback_weight'] *= LIKE_FACTOR
    
    if not disliked_indices.empty:
        new_pool.loc[disliked_indices, 'feedback_weight'] *= DISLIKE_FACTOR
        
    # Recalculer le poids final
    new_pool['final_sampling_weight'] = new_pool['composite_score'] * new_pool['feedback_weight']
    
    return new_pool

# --- Simulation d'une boucle interactive ---

# 1. L'utilisateur voit 'sample_1'.
# Supposons que l'utilisateur 'like' le motif à l'index 0 de l'échantillon
# et 'dislike' le motif à l'index 2.

# (Attention : ce sont les index de l'échantillon, il faut retrouver 
# les index du pool P_sampling)
liked_rules_indices = sample_1.index[0:1] # Simule "like" sur le premier
disliked_rules_indices = sample_1.index[2:3] # Simule "dislike" sur le troisième

print(f"\nFeedback: LIKE sur index {liked_rules_indices.tolist()}, DISLIKE sur index {disliked_rules_indices.tolist()}")

# 2. Mise à jour des poids dans le pool principal 
P_sampling_updated = update_weights_with_feedback(P_sampling, 
                                                  liked_rules_indices, 
                                                  disliked_rules_indices)

print("\nAperçu des poids mis à jour dans le pool P :")
# Affichage des motifs qui ont été modifiés
changed_indices = list(liked_rules_indices) + list(disliked_rules_indices)
print(P_sampling_updated.loc[changed_indices][['antecedents', 'consequents', 'composite_score', 'feedback_weight', 'final_sampling_weight']])


# 3. Génération d'un NOUVEL échantillon (deuxième boucle)
# Le nouvel échantillon devrait refléter les préférences.
sample_2 = weighted_sampling(P_sampling_updated, k=k_sample, with_replacement=False)

print(f"\n--- Deuxième échantillon (k={k_sample}) après feedback ---")
print(sample_2[['antecedents', 'consequents', 'composite_score', 'final_sampling_weight']])



Feedback: LIKE sur index [18321], DISLIKE sur index [10981]

Aperçu des poids mis à jour dans le pool P :
             antecedents    consequents  composite_score  feedback_weight  \
18321  (76, 28, 127, 39)  (41, 62, 126)         0.702869              1.5   
10981     (109, 127, 70)           (41)         0.447944              0.5   

       final_sampling_weight  
18321               1.054303  
10981               0.223972  

--- Deuxième échantillon (k=3) après feedback ---
                  antecedents consequents  composite_score  \
18183    (76, 28, 29, 39, 41)  (127, 126)         0.625893   
26821    (70, 62, 93, 39, 41)        (76)         0.548696   
14585  (70, 29, 127, 126, 41)        (76)         0.548696   

       final_sampling_weight  
18183               0.625893  
26821               0.548696  
14585               0.548696  
